---
preview: LangChain support for HANA KG engine

In [ ]:
# !pip install ../whl/langchain_hana-0.1.0-py3-none-any.whl

In [ ]:
import langchain
# import langchain_community
import langchain_hana
print('langchain version:', langchain.__version__)
# print('langchain_community version:', langchain_community.__version__)
print('langchain_hana version:', langchain_hana.__version__)

In [ ]:
from langchain_hana import HanaRdfGraph
from langchain_hana import HanaSparqlQAChain

In [ ]:
# connect to HANA Cloud
from hdbcli import dbapi
connection = dbapi.connect(key='KGE2')

---

In [ ]:
# let's create a LangChain Graph Store by referring to a named graph and its ontology
ontology_uri = "wiki_movies_ontology"
graph_uri = "wiki_movies"

hkg = HanaRdfGraph(
    connection=connection, 
    ontology_uri=ontology_uri, 
    graph_uri=graph_uri
    )

# print(hkg.get_schema)


In [ ]:
# sure we can run SPARQL queries
sparql_query = '''
	PREFIX demo: <http://kg.demo.sap.com/>
	PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	SELECT ?movie ?actor ?actorLabel
	FROM <wiki_movies>
	WHERE {
		?movie a demo:Film ;
	  		demo:title "The Matrix" .
		?actor demo:acted_in ?movie ;
	  		rdfs:label ?actorLabel .
	} LIMIT 10
    '''

# hkg.query(query=sparql_query,content_type='application/sparql-results+json')

hkg.convert_csv_response_to_dataframe(hkg.query(query=sparql_query))

---
Create a LangChain SPARQL QA Chain<br>
The SPARQL QA Chain translates a natural language query into a SPARQL query which fits the KGs model

In [ ]:
# we need an LLM to generate the SAPRQL queries
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
llm = ChatOpenAI(
    proxy_model_name="gpt-4o", temperature=0
)

In [ ]:
# let's create a SPARQL QA Chain
hkg_qa_chain = HanaSparqlQAChain.from_llm(
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    graph=hkg
    )

In [ ]:
query = "Which directors played most often a role in their film? Which roles did they play and what are the film titles?"
# query = "which are the two latest film in the genre 'action film' and who directed them?"

output = hkg_qa_chain.invoke(query)

In [ ]:
print('User input: ' + output['query'])
print('LLM response: ' + output['result'])

---

In [ ]:
# sure we can run SPARQL queries
sparql_query = '''
PREFIX kg: <http://kg.demo.sap.com/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT *

FROM <wiki_movies>
WHERE {
    ?director a kg:Director ;
              kg:directed ?film ;
              rdfs:label ?directorName .
    
    ?film kg:publicationDate ?pubDate ;
          kg:title ?filmTitle ;
          kg:castMemberStatement ?castStatement .
    ?castStatement kg:castMember ?director ;
                   kg:characterRole ?role .
    FILTER (YEAR(?pubDate) = 2022)
}'''
hkg.convert_csv_response_to_dataframe(hkg.query(query=sparql_query))
